In [ ]:
import pandas as pd
import xarray as xr
xr.set_options(keep_attrs=True,
               display_expand_data=False)

import plotly.express as px

from risk_data import get_factor_data
from risk_config_port import portfolios
import risk_portfolios as p
from risk_chart import px_format
from risk_chart_port import draw_portfolio_cumret, draw_portfolio_weights, get_portfolio_summary

from importlib import reload
reload(p)



# Initialize parameters

In [ ]:
factor_data = get_factor_data()
returns = factor_data.ret.to_pandas()

# tickers = [ 'MWTIX', 'SPY', 'IWM', 'MDY', 'RSP', 'QQQ', 'XLK', 'XLI', 'XLF', 'XLC', 'XLE', 'XLY', 'XLB', 'XLV', 'XLU', 'XLP', 'VNQ', 'AIQ', 'ICLN', 'PFF', 'FEZ', 'EEM', 'FXI', 'ASHR',  'LQD', 'HYG', 'LQDH', 'HYGH', 'AGG',  'SHY', 'IEI', 'IEF', 'TLT', 'TIP', 'VTIP', 'AGNC', 'VMBS', 'CMBS', 'EMB', 'EMHY', 'GLD', 'SLV', 'USO', 'DBC', 'UUP', 'FXE', 'FXY' ]
# returns_lm = p.get_returns_basic(tickers)
rebalancing_dates = returns.resample('M').last().index


# Run portfolio construction

In [ ]:
portfolio_tuple = p.build_all_portfolios(portfolios, returns, rebalancing_dates)
returns, portfolio_weights_long = portfolio_tuple

# Observe output

In [ ]:
display(get_portfolio_summary(returns).style)
draw_portfolio_cumret(returns).show()
draw_portfolio_weights(portfolio_weights_long)
